In [136]:
%pylab inline
import pandas as pd
import seaborn
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics.pairwise import manhattan_distances

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['unique']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [137]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')
import cv2

In [138]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

## TRAINING TIME!

In [139]:
# new_df.to_json('/Users/Home/Desktop/CMPT 459/MI2/train_fe.json')
train_df = pd.read_json(r'C:\Users\Giovanni PC\cmpt459\data\ml3\final_train_df_m3.json')

# new_df.to_json('/Users/Home/Desktop/CMPT 459/MI2/train_fe.json')
test_df = pd.read_json(r'C:\Users\Giovanni PC\cmpt459\data\final_test_df_m3.json')


In [140]:
train_df.head()
test_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,...,created_dt,managerID_count,buildingID_count,weekends,month_4,month_5,created_nights,created_morning,created_afternoon,bed_bath_ratio
0,1.0,1,79780be1514f645d7e6be99a3de696c5,2016-06-11 05:29:41,Large with awesome terrace--accessible via bed...,Suffolk Street,"[Elevator, Laundry in Building, Laundry in Uni...",40.7185,7142618,-73.9865,...,1465622981000,38,5,1,0,0,1,0,0,1.0
1,1.0,2,0,2016-06-24 06:36:34,Prime Soho - between Bleecker and Houston - Ne...,Thompson Street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7278,7210040,-74.0000,...,1466750194000,17,12378,1,0,0,0,1,0,2.0
2,1.0,0,0,2016-06-17 01:23:39,Spacious studio in Prime Location. Cleanbuildi...,Sullivan Street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7260,7174566,-74.0026,...,1466126619000,3854,12378,1,0,0,1,0,0,0.0
3,1.0,2,f9c826104b91d868e69bd25746448c0c,2016-06-21 05:06:02,For immediate access call Bryan.<br /><br />Bo...,Jones Street,"[Hardwood Floors, Dogs Allowed, Cats Allowed]",40.7321,7191391,-74.0028,...,1466485562000,176,10,0,0,0,1,0,0,2.0
4,1.0,1,81062936e12ee5fa6cd2b965698e17d5,2016-06-16 07:24:27,Beautiful TRUE 1 bedroom in a luxury building ...,Exchange Place,"[Roof Deck, Doorman, Elevator, Fitness Center,...",40.7054,7171695,-74.0095,...,1466061867000,35,94,0,0,0,0,1,0,1.0


In [141]:
train_df.describe()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48329 entries, 0 to 48328
Data columns (total 46 columns):
bathrooms            48329 non-null float64
bedrooms             48329 non-null int64
building_id          48329 non-null object
created              48329 non-null object
description          48329 non-null object
display_address      48329 non-null object
features             48329 non-null object
latitude             48329 non-null float64
listing_id           48329 non-null int64
longitude            48329 non-null float64
manager_id           48329 non-null object
photos               48329 non-null object
price                48329 non-null int64
street_address       48329 non-null object
interest_level       48329 non-null object
dt_distance          48329 non-null float64
pet_policy           48329 non-null int64
listing_type         48329 non-null int64
unit_features        48329 non-null int64
building_features    48329 non-null int64
apartment            48329 non-nul

In [142]:
unique, counts = np.unique(train_df.manager_id, return_counts=True)

id_dict = np.asarray((unique,counts)).T
id_dict

array([['0000abd7518b94c35a90d64b56fbf3e6', 12],
       ['001ce808ce1720e24a9510e014c69707', 10],
       ['003fc4e9a70053082f131b1054966aaf', 1],
       ...,
       ['ffb9eceeaa4d60da7b41cc8f0954cd50', 2],
       ['ffd3c60281949a5cd806030203a79558', 6],
       ['ffd7307e4924587b30ec014a95db1362', 115]], dtype=object)

In [143]:
id_dic = {}
for val in id_dict:
    id_dic[val[0]] = val[1]

In [144]:
max(id_dic.values())

2471

In [145]:
v1_selected_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "pet_policy", "listing_type", 
                     "unit_features", "building_features", "apartment", "appliance", "bathroom", "bedroom",
                     "building", "contact", "floor", "high", "kitchen", "room", "num_photos", 
                     "description_len", "mall_distance", "dt_distance"]

In [146]:
X = train_df[v1_selected_feats]
y = train_df["interest_level"]

In [147]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [148]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

In [149]:
X

array([[0.1       , 0.125     , 0.27075252, ..., 0.11411411, 0.19559873,
        0.1097247 ],
       [0.1       , 0.25      , 0.34930178, ..., 0.1951952 , 0.05667761,
        0.11368588],
       [0.1       , 0.25      , 0.36132661, ..., 0.17717718, 0.07044386,
        0.14517726],
       ...,
       [0.1       , 0.125     , 0.01027929, ..., 0.12012012, 0.36704753,
        0.37274708],
       [0.1       , 0.25      , 0.33669511, ..., 0.17567568, 0.02552656,
        0.04634581],
       [0.1       , 0.375     , 0.36501164, ..., 0.08558559, 0.03237035,
        0.13170925]])

In [150]:
#Train for decision Tree
test_scores = []
train_scores = []
log_scores = []
log_scores_train = []

min_sample_arr = [1, 50, 100, 200, 500] 

i = 0
cv = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in cv.split(X):
    #min samples default 2, min samples leaf default =1. min weight frac =0 , max feat = 4
    rfc  = RandomForestClassifier(n_estimators = 100, criterion = "gini", max_leaf_nodes = 4, max_features='auto', random_state=0, min_samples_leaf=1, min_samples_split = 2)
    i += 1
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y.iloc[train_index], y.iloc[test_index]
    rfc.fit(X_train, y_train)
    test_scores.append(rfc.score(X_test, y_test))
    train_scores.append(rfc.score(X_train, y_train))
    y_test_pred = rfc.predict_proba(X_test)
    y_train_pred = rfc.predict_proba(X_train)
    log_scores.append(log_loss(y_test, y_test_pred))
    log_scores_train.append(log_loss(y_train, y_train_pred))
    
    #when the min sample change, there is pverfitting for small values,
    #as the calues grow, the training set does progressively worse in comparison to the test 
    # use 50 as the number
    

In [151]:
log_scores


[0.7433498727383955,
 0.7437944579951052,
 0.7454356471530916,
 0.7563410252505085,
 0.7424130131487622]

In [153]:
log_scores_train

[0.7465511564118881,
 0.7466381089869191,
 0.7463127049513092,
 0.7422223159818396,
 0.7467515889480466]

In [154]:
y_train_pred


array([[0.09514738, 0.63270366, 0.27214896],
       [0.0688304 , 0.68772551, 0.24344409],
       [0.07310354, 0.69754842, 0.22934805],
       ...,
       [0.09778623, 0.64811341, 0.25410036],
       [0.07213606, 0.67593527, 0.25192867],
       [0.07160578, 0.6789583 , 0.24943592]])

In [155]:
labels2idx = {label: i for i, label in enumerate(rfc.classes_)}
labels2idx

{'high': 0, 'low': 1, 'medium': 2}

In [156]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

In [157]:
rfc_test  = RandomForestClassifier(n_estimators = 100, criterion = "gini", min_weight_fraction_leaf = 0.2, max_leaf_nodes = 4, min_samples_split = 5, max_features='sqrt', random_state=0, min_samples_leaf=200, max_depth=10)
rfc_test.fit(X_train, y_train)
y_val_pred = rfc_test.predict_proba(X_val)
log_loss(y_val, y_val_pred)

0.7537721428906327

In [158]:
test_X = test_df[v1_selected_feats]

In [159]:
test_X = scaler.fit_transform(test_X)

In [160]:
test_y = rfc_test.predict_proba(test_X)

In [161]:
labels2idx = {label: i for i, label in enumerate(rfc_test.classes_)}
labels2idx

{'high': 0, 'low': 1, 'medium': 2}

In [162]:
sub = pd.DataFrame()
sub["listing_id"] = test_df["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = test_y[:, labels2idx[label]]
sub.to_csv(r"C:\Users\Giovanni PC\cmpt459\data\submission_m3_rf_v1.csv", index=False)
